## Requests

O que é um **request**?
Um request é um pedido de informação, normalmente a um website que oferece um serviço. Por exemplo, se você quer saber se um CEP é válido ou a qual endereço ele se refere, existe o webservice <a href="https://viacep.com.br/" target="_blank">Viacep</a>. Por exemplo, se você entrar com o link https://viacep.com.br/ws/36570047/json/ no navegador, ele irá te retornar uma resposta no formado JSON com dados referentes ao endereço do CEP 3657-047.


Vamos começar importando a biblioteca python <a href="https://requests.readthedocs.io/en/master/" target="_blank">**requests**</a>:

In [5]:
import requests

Feito isso, vamos criar a função `achaEndereco` que recebe um CEP e retorna um endereço como resposta:

In [6]:
def achaEndereco(cep):
    resposta = requests.get('http://viacep.com.br/ws/' + cep + '/json/')
    resposta = resposta.json()

    return resposta

Agora, quando executamos a função enviando um CEP como parâmetro temos o endereço referente ao CEP como um dicionário python:

In [7]:
achaEndereco('36570047')

{'cep': '36570-047',
 'logradouro': 'Avenida Bueno Brandão',
 'complemento': '',
 'bairro': 'Centro',
 'localidade': 'Viçosa',
 'uf': 'MG',
 'ibge': '3171303',
 'gia': '',
 'ddd': '31',
 'siafi': '5427'}

Através do mesmo serviço também é possível fazer o caminho contrário e encontrar um CEP a partir de um determinado endereço:

In [8]:
def achaCEP(estado, cidade, rua):
    resposta = requests.get('http://viacep.com.br/ws/' + estado + '/' + cidade + '/' + rua + '/json/')
    resposta = resposta.json()

    return resposta

Como é possível que um endereço tenha mais de um CEP (no caso de um endereço incompleto, por exemplo), o serviço retorna uma lista de possíveis endereços:

In [9]:
achaCEP('MG', 'Viçosa', 'Avenida Bueno Brandão')

[{'cep': '36570-047',
  'logradouro': 'Avenida Bueno Brandão',
  'complemento': '',
  'bairro': 'Centro',
  'localidade': 'Viçosa',
  'uf': 'MG',
  'ibge': '3171303',
  'gia': '',
  'ddd': '31',
  'siafi': '5427'},
 {'cep': '36570-970',
  'logradouro': 'Avenida Bueno Brandão 678',
  'complemento': '',
  'bairro': 'Centro',
  'localidade': 'Viçosa',
  'uf': 'MG',
  'ibge': '3171303',
  'gia': '',
  'ddd': '31',
  'siafi': '5427'},
 {'cep': '36571-959',
  'logradouro': 'Avenida Bueno Brandão 678',
  'complemento': '',
  'bairro': 'Centro',
  'localidade': 'Viçosa',
  'uf': 'MG',
  'ibge': '3171303',
  'gia': '',
  'ddd': '31',
  'siafi': '5427'}]

Vamos agora utilizar um outro serviço interessante, o <a href="https://www.graphhopper.com/">**GraphHopper**</a>. Com o GraphHopper é possível encontrar as coordenadas de um local a partir de uma busca pelo endereço, um recurso conhecido como *Geocoding*, entre outros recursos. Antes de tudo é necessário termos uma conta no site pois o serviços requer uma chave de desenvolvedor. Para as finalidades deste exercício, usarei minha própria chave. Para facilitar o uso da chave do código, vamos declará-la no código como uma constante:

In [10]:
GRAPHHOPPER_API_KEY = 'f9e77954-a5fd-416e-8087-1c1c456c6fc8'

Vamos agora criar a função `achaCoord`, que recebe como parâmetro uma string de busca (um endereço) e retorna a coordenada mais provável para este endereço:

In [11]:
def achaCoord(endereco):
    locale = 'br'
    url = 'https://graphhopper.com/api/1/geocode?q=' + endereco + '&locale=' + locale + '&key=' + GRAPHHOPPER_API_KEY
    resposta = requests.get(url)
    resposta = resposta.json()

    return (resposta['hits'][0]['point']['lat'], resposta['hits'][0]['point']['lng'])

Para entendermos como funciona essa função é preciso dar uma olhada na <a href="https://docs.graphhopper.com/#tag/Geocoding-API" target="_blank">documentação</a> do serviço.
A URL, ou seja, o endereço web do serviço, é sempre 'https://graphhopper.com/api/1/geocode?', seguida de seus parâmetros, separados por '&': `q=query_de_busca`, `locale=br` para endereços localizados no Brasil e `key=chave`. O serviço retorna um JSON (convertido para um dicionário em python) com as chaves _hits_, que é uma lista de prováveis coordenadas, e _took_, que é o tempo que o serviço demorou para encontrar os resultados. Cada _hit_ encontrado tem é formado de diversos dados sobre o endereço e, entre eles, um ponto, que é um par de coordenadas (lat, lng), que é o que a função retorna:

In [12]:
casa = achaCoord('Avenida Bueno Brandão, Viçosa - MG')

In [13]:
casa

(-20.7547399, -42.8820165)

Agora vamos um pouquinho mais longe e usar uma outra biblioteca python para encontrar essa coordenada em um mapa. A biblioteca <a href="https://python-visualization.github.io/folium/" target="_blank">**Folium**</a> pode nos auxiliar nisso:

In [14]:
import folium

A biblioteca **Folium** é um encapsulamento para que a biblioteca Javascript <a href="https://leafletjs.com/" target="_blank">**Leaflet**</a> possa ser usada em python. É uma ferramenta que permite utilizar diversos serviços de mapas, ou seja, você pode utilizar o serviços livre <a href="https://www.openstreetmap.org/" target="_blank">**OpenStreetMap** (OSM)</a> para desenhar seu mapa, mas também pode utilizar serviços pagos como <a href="https://www.mapbox.com/" target="_blank">**Mapbox**</a>. Neste exemplo vamos utilizar o OSM, que é o padrão.

Para criar um mapa com o Folium basta uma linha de código:

In [15]:
m = folium.Map(location=casa, zoom_start=15)

A linha acima cria um mapa centralizado na coordenada da variável `casa`. E com isso temos:

In [16]:
m

Usando o código abaixo é possível adicionar um marcador na coordenada encontrada anteriormente:

In [17]:
folium.Marker(casa, icon=folium.Icon(icon='home', color='blue', prefix='fa')).add_to(m)
m

E se eu quiser desenhar no mapa o caminho de casa até a Univiçosa? Utilizando essas duas ferramentas isso é possível. Vamos criar uma função `achaCaminho(a, b)`, que recebe duas coordenadas e retorna o caminho entre elas:

In [20]:
def achaCaminho(a, b):
    coords = 'point=' + str(a[0]) + ',' + str(a[1]) + '&point=' + str(b[0]) + ',' + str(b[1])
    params = 'instructions=false&points_encoded=true&type=json&debug=true'
    url = 'https://graphhopper.com/api/1/route'
    route = requests.get(url + '?' + coords + '&' + params + '&key=' + GRAPHHOPPER_API_KEY)
    route = route.json()

    return route['paths'][0]['points']

Para entender melhor a função acima é preciso olhar a documentação da API da ferramenta GraphHopper, mas explicando de maneira simples, a linha `return route['paths'][0]['points']` retorna os pontos 

In [29]:
# Primeiro vamos inicializar novamente um mapa centralizado nas coordenadas de casa
m = folium.Map(location=casa, zoom_start=13)

# Para facilitar, estas são as coordenadas da Univiçosa
univicosa = (-20.728718, -42.865061)

# Adicionar um marcador em casa e outro na univiçosa
folium.Marker(casa, icon=folium.Icon(icon='home', color='blue', prefix='fa')).add_to(m)
folium.Marker(univicosa, icon=folium.Icon(icon='graduation-cap', color='black', prefix='fa')).add_to(m)

# Usando a função que escrevemos...
caminho = achaCaminho(casa, univicosa)

# Aqui decodificamos os pontos para uma linha e a adicionamos ao mapa
import polyline
locations = polyline.decode(caminho)
folium.PolyLine(locations).add_to(m)

# Agora é só imprimir
m